**Vanessa Navarro Coronado e Iván Sánchez Castellanos**

# Práctica 2 - Clasificación supervisada en scikit-learn

Como siempre, lo primero que hacemos es importar los paquetes necesarios, e inicializar nuestra semilla para nuestros experimentos.

In [1]:
# Always load all scipy stack packages
import numpy as np
import pandas as pd
from scipy import stats, integrate
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
sns.set(color_codes=True)

In [2]:
# This code configures matplotlib for proper rendering
%matplotlib inline
mpl.rcParams["figure.figsize"] = "8, 4"
import warnings
warnings.simplefilter("ignore")

In [3]:
seed=6342
np.random.seed(6342)

### Cargado de los datos

A continuación, cargamos los datos de los datasets Pima y Wisconsin. Cada uno será separado en dos conjuntos, uno con los atributos y otro con la clase del dataset. Esto es necesario para que la libreria scikit learn pueda ser utilizada.

In [4]:
# Update the file path to fit your system
dfPima = pd.read_csv("../data/pima.csv", dtype={ "label": 'category'})
dfAttributesPima = dfPima.drop('label', 1)
dfLabelPima = dfPima['label']

dfWisc = pd.read_csv("../data/wisconsin.csv", dtype={ "label": 'category'})
dfAttributesWisc = dfWisc.drop('label', 1)
dfLabelWisc = dfWisc['label']

Una vez cargados los datasets, debemos separarlos en dos conjuntos de Train y Test para poder realizar nuestros experimentos.

In [5]:
# Divide into train/test split for our experiments
from sklearn.model_selection import train_test_split
train_attsPima, test_attsPima, train_labelPima, test_labelPima = train_test_split( 
    dfAttributesPima,
    dfLabelPima,
    test_size=0.2,
    random_state=seed,
    stratify=dfLabelPima)

train_attsWisc, test_attsWisc, train_labelWisc, test_labelWisc = train_test_split( 
    dfAttributesWisc,
    dfLabelWisc,
    test_size=0.2,
    random_state=seed,
    stratify=dfLabelWisc)

# 1. Selección y evaluación de modelos

In [6]:
from sklearn.model_selection import cross_val_score

## 1.1. Árbol de decisión con preprocesamiento de los datos durante la validación cruzada (transformers y pipelines)

In [7]:
from sklearn.preprocessing import Imputer
# Cargamos el arbol de decision
from sklearn import tree

from sklearn.model_selection import GridSearchCV
import sklearn.metrics as metrics

### 1.1.1. Preprocesamiento de los datos

Para poder realizar el tratamiento de valores perdidos en el dataset Pima, primero debemos seleccionar los atributos que contengan valores a cero que son considerados como perdidos. En este caso nosotros pensamos que un cero en la variable 'preg' no sería valor perdido, ya que los pacientes puede que no hayan estado embarazados ninguna vez, o que sean de género masculino. Por tanto, un cero en cualquiera de las otras variables predictoras será considerado como valor perdido, a excepción de la variable 'preg'.

In [8]:
aux_ceros = dfPima.columns.drop(["preg","label"])
aux_ceros

Index(['plas', 'pres', 'skin', 'insu', 'mass', 'pedi', 'age'], dtype='object')

Una vez separadas las variables sobre las que queremos hacer tratamiento de valores perdidos, reemplazamos los valores a cero por valores de tipo NaN, para que nuestro 'imputer' sea capaz de cambiar estos valores por la media de las variables para nuestro dataset.

In [9]:
for i in aux_ceros:
    train_attsPima.replace({i: {0: np.nan}}, inplace = True)
    train_labelPima.replace({i: {0: np.nan}}, inplace = True)
    test_attsPima.replace({i: {0: np.nan}}, inplace = True)
    test_labelPima.replace({i: {0: np.nan}}, inplace = True)

### 1.1.2. GridSearch

Aquí definimos nuestro GridSearch con los parámetros elegidos. En nuestro caso hemos elegido los parámetros 'criterion' (puede ser "gini", que usa la impureza Gini como función, o “entropy”, usando la ganancia de información), 'max_depth' (para indicar la máxima profundidad del árbol), y 'min_samples_leaf' (para indicar el número mínimo de ejemplos que queremos que haya en cada hoja del árbol).

In [96]:
clfTree = GridSearchCV(
    estimator = tree.DecisionTreeClassifier(),
    param_grid =
        {'criterion': ["entropy","gini"],'max_depth': [3, 5, 10, None], 'min_samples_leaf': [3,5,10]},
    scoring = 'accuracy',
    cv = 10
)

#fitted = clfTree.fit(train_atts, train_label)

Si asignamos el valor 'None' al parámetro 'max_depth', el clasificador intentará llegar a la máxima profundidad del árbol por defecto.

### 1.1.3. Pipeline

A continuación creamos un Pipeline con dos pasos: el primero es imputar los valores perdidos siguiendo la estrategia de rellenar con la media, y el segundo paso es el GridSearch que configuramos en la celda anterior.

In [97]:
from sklearn.pipeline import Pipeline

In [98]:
# We define the pipeline as a set of tuples
estimatorTree = Pipeline([("imputer", Imputer(missing_values='NaN',
                                          strategy="mean",
                                          axis=0)),
                      ("GridSearchTree", clfTree)])

### 1.1.4. Pima

Después de crear el Pipeline, lo ejecutamos con el dataset Pima e imprimimos el accuracy obtenido con la mejor configuración de parámetros del árbol encontrada por nuestro GridSearch.

In [102]:
# We can fit and use the pipeline as usual
clsTreePima = estimatorTree.fit(train_attsPima, train_labelPima)
predictionTreePima = estimatorTree.predict(test_attsPima)
metrics.accuracy_score(test_labelPima, predictionTreePima)


0.74675324675324672

Otra posible forma de evaluar el Pipeline generado, utilizando la validación cruzada.

In [103]:
# Or we can use it as the input of a cross-val scorer
cross_val_score(estimatorTree, train_attsPima, train_labelPima).mean()

0.71335427374833438

Por último, nos parece interesante mostrar la mejor configuración de los parámetros del árbol que ha encontrado nuestro GridSearch.

In [104]:
clsTreePima.named_steps['GridSearchTree'].best_params_

{'criterion': 'entropy', 'max_depth': 3, 'min_samples_leaf': 3}

Además del accuracy, también es interesante mostrar la matriz de confusión obtenida, y las medidas de precision y recall.

In [105]:
metrics.confusion_matrix(test_labelPima, predictionTreePima)

array([[85, 15],
       [24, 30]])

In [106]:
# Recall
metrics.recall_score(test_labelPima, predictionTreePima, pos_label="tested_positive")

0.55555555555555558

In [107]:
# Precision
metrics.precision_score(test_labelPima, predictionTreePima, pos_label="tested_positive")

0.66666666666666663

### 1.1.5. Wisconsin

In [108]:
# We can fit and use the pipeline as usual
clsTreeWisc = estimatorTree.fit(train_attsWisc, train_labelWisc)
predictionTreeWisc = estimatorTree.predict(test_attsWisc)
metrics.accuracy_score(test_labelWisc, predictionTreeWisc)


0.93571428571428572

In [109]:
# Or we can use it as the input of a cross-val scorer
cross_val_score(estimatorTree, train_attsWisc, train_labelWisc).mean()

0.93384509228911516

In [110]:
clsTreeWisc.named_steps['GridSearchTree'].best_params_

{'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 3}

Matriz de confusión, precision y recall.

Además del accuracy, también es interesante mostrar la matriz de confusión obtenida, y las medidas de precision y recall.

In [111]:
metrics.confusion_matrix(test_labelWisc, predictionTreeWisc)

array([[90,  2],
       [ 7, 41]])

In [112]:
# Recall
metrics.recall_score(test_labelWisc, predictionTreeWisc, pos_label="malignant")

0.85416666666666663

In [113]:
# Precision
metrics.precision_score(test_labelWisc, predictionTreeWisc, pos_label="malignant")

0.95348837209302328

## 1.2. KNN con preprocesamiento de los datos durante la validación cruzada (transformers y pipelines)

In [28]:
from sklearn import neighbors

In [29]:
import sklearn.metrics as metrics

In [30]:
from sklearn.model_selection import GridSearchCV

### 1.2.1. GridSearch

In [167]:
clfKNN = GridSearchCV(
    estimator = neighbors.KNeighborsClassifier(),
    param_grid = 
        { 'n_neighbors' : [1,2,3,4,5] },
    scoring = 'accuracy',
    cv = 10
)

#fitted = clfKNN.fit(train_atts, train_label)

### 1.2.2. Pipeline

In [168]:
# We define the pipeline as a set of tuples
estimatorKNN = Pipeline([("imputer", Imputer(missing_values='NaN',
                                          strategy="mean",
                                          axis=0)),
                      ("GridSearchKNN", clfKNN)])

### 1.2.3. Pima

In [169]:
# We can fit and use the pipeline as usual
clsKNNPima = estimatorKNN.fit(train_attsPima, train_labelPima)
predictionKNNPima = estimatorKNN.predict(test_attsPima)
metrics.accuracy_score(test_labelPima, predictionKNNPima)

0.68831168831168832

In [170]:
# Or we can use it as the input of a cross-val scorer
cross_val_score(estimatorKNN, train_attsPima, train_labelPima).mean()

0.75899486007995431

In [171]:
clsKNNPima.named_steps['GridSearchKNN'].best_params_

{'n_neighbors': 5}

Matriz de confusión, precision y recall obtenidos.

In [172]:
metrics.confusion_matrix(test_labelPima, predictionKNNPima)

array([[82, 18],
       [30, 24]])

In [173]:
# Recall
metrics.recall_score(test_labelPima, predictionKNNPima, pos_label="tested_positive")

0.44444444444444442

In [174]:
# Precision
metrics.precision_score(test_labelPima, predictionKNNPima, pos_label="tested_positive")

0.5714285714285714

### 1.2.4. Wisconsin

In [175]:
# We can fit and use the pipeline as usual
clsKNNWisc = estimatorKNN.fit(train_attsWisc, train_labelWisc)
predictionKNNWisc = estimatorKNN.predict(test_attsWisc)
metrics.accuracy_score(test_labelWisc, predictionKNNWisc)

0.94999999999999996

In [176]:
# Or we can use it as the input of a cross-val scorer
cross_val_score(estimatorKNN, train_attsWisc, train_labelWisc).mean()

0.93741015467770694

In [177]:
clsKNNWisc.named_steps['GridSearchKNN'].best_params_

{'n_neighbors': 1}

Matriz de confusión, precision y recall obtenidos.

In [178]:
metrics.confusion_matrix(test_labelWisc, predictionKNNWisc)

array([[91,  1],
       [ 6, 42]])

In [179]:
metrics.accuracy_score(test_labelWisc, predictionKNNWisc)

0.94999999999999996

In [180]:
# Recall
metrics.recall_score(test_labelWisc, predictionKNNWisc, pos_label="malignant")

0.875

In [181]:
# Precision
metrics.precision_score(test_labelWisc, predictionKNNWisc, pos_label="malignant")

0.97674418604651159

# 2. Implementación de GridSearch manualmente

In [236]:
'''clf = GridSearchCV(
    estimator = tree.DecisionTreeClassifier(),
    param_grid =
        {'criterion': ["entropy","gini"],'max_depth': [3, 5, 10, None], 'min_samples_leaf': [3,5,10]},
    scoring = 'accuracy',
    cv = 10
)'''
def GridS(estim, paramG, score, crossval):
    for atrib in paramG.keys():
        for value in paramG[atrib]:
            estim.set_params(**{atrib:value})
            print(estim.get_params())

## 2.1. Árbol de decisión con GridSearch manual

In [237]:
s=GridS(
    estim = tree.DecisionTreeClassifier(),
    paramG={'min_samples_leaf': [3,5,10]},
    score=None,
    crossval=None
)
s

{'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'presort': False, 'random_state': None, 'splitter': 'best'}
{'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 5, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'presort': False, 'random_state': None, 'splitter': 'best'}
{'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 10, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'presort': False, 'random_state': None, 'splitter': 'best'}


In [231]:
estim=tree.DecisionTreeClassifier()

In [235]:
estim.set_params(**{'min_samples_leaf': 5})

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')